In [ ]:
! pip install -U weaviate-client

In [16]:
# import weaviate
# from weaviate.classes.init import Auth 

# # Best practice: store your credentials in environment variables
# weaviate_url = 'https://dbvzpt0resxqjgufx5ing.c0.us-west3.gcp.weaviate.cloud'
# weaviate_api_key = '3yci5k3foVl0BOIJIPmgv9u6SCySehkn15AY'

# # Connect to Weaviate Cloud
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=weaviate_url, 
#     auth_credentials=Auth.api_key(weaviate_api_key),
# )

# print(client.is_ready()) 

True


In [1]:
import weaviate

headers = {
    "X-Zhipuai-Api-Key": '4d6c5f8ad43b8fff94480b95a5a6c5d0.ouFQYxQ8pIVKO7J8'
}
client = weaviate.connect_to_local(headers=headers)

print(client.is_ready())

True


D:\project\weaviate-python-client\weaviate\warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 0.1.dev3116+geac93c3. The latest version is 4.10.2.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [11]:
# client.collections.delete("OpenVid_1M") 


In [13]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances
 
client.collections.create(
    "OpenVid_1M",
    reranker_config=Configure.Reranker.transformers(),
    generative_config=Configure.Generative.zhipuai(
        # These parameters are optional
        model="glm-4",   
        max_tokens=500, 
        temperature=0.7,
        top_p=0.7
    ),
    vectorizer_config=[
        # Set a named vector
        Configure.NamedVectors.text2vec_transformers(   
            name="Video", source_properties=["Video"] # 视频路径
        ),
        # Set another named vector
        Configure.NamedVectors.text2vec_transformers(   
            name="Caption", source_properties=["Caption"], # 视频描述
            vector_index_config=Configure.VectorIndex.hnsw(
                distance_metric=VectorDistances.COSINE
            )
        ),
        # Set another named vector
        Configure.NamedVectors.text2vec_transformers(  
            name="AestheticScore", source_properties=["AestheticScore"]        # 审美分数
        ),
        # Set another named vector
        Configure.NamedVectors.text2vec_transformers(   
            name="MotionScore", source_properties=["MotionScore"] # 运动得分 
           
        ),
        Configure.NamedVectors.text2vec_transformers(   
            name="TemporalConsistencyScore", source_properties=["TemporalConsistencyScore"] # 时间一致得分 
      
        ),
        Configure.NamedVectors.text2vec_transformers(   
            name="CameraMotion", source_properties=["CameraMotion"] # 相机运动
       
        ),
        Configure.NamedVectors.text2vec_transformers(   
            name="Frame", source_properties=["Frame"] # 帧
        
        ),
        Configure.NamedVectors.text2vec_transformers(   
            name="FPS", source_properties=["FPS"] # fps
         
        ),
        Configure.NamedVectors.text2vec_transformers(  
            name="Seconds", source_properties=["Seconds"] # 时长
        )
    ],
 
    properties=[  # Define properties
        Property(name="Video", data_type=DataType.TEXT),
        Property(name="Caption", data_type=DataType.TEXT),
        Property(name="AestheticScore", data_type=DataType.NUMBER),
        Property(name="MotionScore", data_type=DataType.NUMBER),
        Property(name="TemporalConsistencyScore", data_type=DataType.NUMBER),
        Property(name="CameraMotion", data_type=DataType.TEXT),
        Property(name="Frame", data_type=DataType.NUMBER),
        Property(name="FPS", data_type=DataType.NUMBER),
        Property(name="Seconds", data_type=DataType.NUMBER)
        
    ],
)

In [2]:
 
import os
import requests

file_path = "OpenVid-1M.csv"

# 检查文件是否存在，如果不存在则下载
if not os.path.exists(file_path):
    print("文件不存在，正在下载...")
    response = requests.get(
        "https://huggingface.co/datasets/nkp37/OpenVid-1M/resolve/main/data/train/OpenVid-1M.csv"
    )
    # 保存文件
    with open(file_path, "wb") as f:
        f.write(response.content)
    print("文件下载完成！")
else:
    print("文件已存在，无需下载。")

# 确认文件状态
file_exists = os.path.exists(file_path)
file_exists
 

文件已存在，无需下载。


True

In [ ]:
import pandas as pd
# Settings for displaying the import progress
counter = 0
interval = 200  # print progress every this many records; should be bigger than the batch_size
collection = client.collections.get("OpenVid_1M")

print("JSON streaming, to avoid running out of memory on large files...")
  
 
# 分块读取 CSV 文件
chunk_size = 10000   # 每次读取10000行
for idx, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    print(f"Processing chunk {idx + 1}...")
    for _, row in chunk.iterrows():
        if idx<1:
            continue
        # 读取每一行的视频信息和标题等
        video_filename = row['video']
        caption = row['caption']
        aesthetic_score = row['aesthetic score']
        motion_score = row['motion score']
        temporal_score = row['temporal consistency score']
        camera_motion = row['camera motion']
        frame = row['frame']
        fps = row['fps']
        seconds = row['seconds']
 
        # 构建数据
        properties = {
            "Video": video_filename,
            "Caption": caption,
            "AestheticScore": aesthetic_score,
            "MotionScore": motion_score,
            "TemporalConsistencyScore": temporal_score,
            "CameraMotion": camera_motion,
            "Frame": frame,
            "FPS": fps,
            "Seconds": seconds
        }

        # 向量数据库插入数据
        collection.data.insert(
            properties=properties, 
        )

        counter += 1

        # 每处理200条数据输出一次进度
        if counter % interval == 0:
            print(f"Imported {counter} records...")
            
# 完成后输出总进度
print(f"Finished importing {counter} records.")

JSON streaming, to avoid running out of memory on large files...
Processing chunk 1...
Processing chunk 2...
Imported 200 records...
Imported 400 records...


In [2]:

collection_shards = collection.config.get_shards()
print(collection_shards)

NameError: name 'collection' is not defined

In [10]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances, ReplicationDeletionStrategy
import weaviate.classes as wvc
from weaviate.collections.classes.config import _CollectionConfigUpdate, Reconfigure
collection = client.collections.get("OpenVid_1M")

collection.config.update(  
  reranker_config=Configure.Reranker.transformers(),
  replication_config=wvc.config.Reconfigure.replication(
        factor=1,
        async_enabled=True,  # Enable asynchronous repair
        deletion_strategy=ReplicationDeletionStrategy.TIME_BASED_RESOLUTION,  # Added in v1.28; Set the deletion conflict resolution strategy
  ),
  inverted_index_config= wvc.config.Reconfigure.inverted_index(  # Optional
        bm25_b=0.56,
        bm25_k1=1.25
  ),
  vectorizer_config=[
       
        wvc.config.Reconfigure.NamedVectors.update(
            name="Caption",
            vector_index_config=wvc.config.Reconfigure.VectorIndex.hnsw(
                quantizer=wvc.config.Reconfigure.VectorIndex.Quantizer.pq()
            )
        ) 
    ]
)